In [1]:
import numpy as np

def calcul_covariance_custom(X):
    """
    Calcule la matrice de covariance d'un dataset (N échantillons, D features)
    Sans utiliser np.cov, avec broadcasting et einsum.
    """
    # N = nombre d'échantillons (lignes)
    N = X.shape[0]
    
    # ÉTAPE 1 : Centrage des données (Broadcasting)
    # On calcule la moyenne de chaque colonne (feature)
    moyenne = np.mean(X, axis=0) 
    
    # Broadcasting : On soustrait le vecteur moyenne à CHAQUE ligne de la matrice
    # (N, D) - (D,) -> (N, D)
    X_centre = X - moyenne
    
    # ÉTAPE 2 : Produit matriciel (Einsum)
    # La formule mathématique est : (X_centre.T @ X_centre)
    # Avec Einsum, on décrit les indices :
    # 'ki' : k = échantillon, i = feature 1
    # 'kj' : k = même échantillon, j = feature 2
    # '->ij' : On somme sur k (dimension commune disparait), il reste la grille (i, j)
    matrice_cov = np.einsum('ki,kj->ij', X_centre, X_centre)
    
    # ÉTAPE 3 : Normalisation
    # On divise par N-1 (estimateur non biaisé, comme le défaut de pandas/numpy)
    return matrice_cov / (N - 1)

# --- VÉRIFICATION ---

# Création d'un dataset fictif (100 lignes, 3 features)
np.random.seed(42)
dataset = np.random.rand(100, 3)

# 1. Notre méthode
resultat_custom = calcul_covariance_custom(dataset)

# 2. La méthode officielle (rowvar=False car nos features sont en colonnes)
resultat_numpy = np.cov(dataset, rowvar=False)

print("Matrice de Covariance (Custom Einsum) :")
print(resultat_custom)
print("\nMatrice de Covariance (NumPy Officiel) :")
print(resultat_numpy)

# Preuve mathématique que c'est identique
print(f"\nDifférence maximale : {np.abs(resultat_custom - resultat_numpy).max()}")
# Doit être très proche de 0 (ex: 1e-16)

Matrice de Covariance (Custom Einsum) :
[[ 0.08177146  0.00473089 -0.0073587 ]
 [ 0.00473089  0.09623581 -0.0013255 ]
 [-0.0073587  -0.0013255   0.08300967]]

Matrice de Covariance (NumPy Officiel) :
[[ 0.08177146  0.00473089 -0.0073587 ]
 [ 0.00473089  0.09623581 -0.0013255 ]
 [-0.0073587  -0.0013255   0.08300967]]

Différence maximale : 1.3877787807814457e-17
